In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
def orgs_dic(organismosfile):
    organismos = {}
    tree = ET.parse(organismosfile)
    root = tree.getroot()
    for organismo in root:
        for item in organismo:
            if item.tag == "codOrganismo":
                v = item.text
            if item.tag == "NIF":
                k = item.text
        organismos.update({k:v})
    return organismos

In [3]:
def buscaorg(nif,dic):
    try:
        return dic[f"{nif}"]
    except:
        return nif

In [4]:
def ajustanumeros (numero):
    if len(numero) == 1:
        try:
            int(numero)
            return "0" + numero
        except:
            return numero
    else:
        return str(numero)

In [5]:
archivo_organismos = "C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Data/organismos_CE_TODOS_TODAS_TODOS_PRE.xml"
contratosfile = "C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/Contratos/CONTRATOS_CEUTA.csv"
transaccionesfile = "C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/Ceuta_TB1-3.csv"
transaccionesB2file = "C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/CEU_B2_dec.csv"
transaccionesIPfile = "C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/Transacciones_IP/CEU_CV6_CV40_dec.csv"

In [6]:
def contratos(contratosfile, dictionary):
    root = ET.Element("contratos")
    with open(contratosfile) as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        for row in csv_reader:
            contrato = ET.SubElement(root,"contrato")
            programaoperativo = 0
            fechaslicitacion = 0
            fechasformalizacion = 0
            subcontratistascount = 0 
            utescount = 0
            for k,v in row.items():
                if v == "":
                    pass
                else:
                    v = ajustanumeros(v)
                    k = k.split(" ")[0].lower()
                    if k == "cod_medio_pub" or k == "f_publicacion" or k=="descripcion":
                        if fechaslicitacion == 0:
                            licitaciones = ET.SubElement(contrato,"fechas_publicacion_licitacion")
                            fechaslicitacion = 1
                        if k == "cod_medio_pub":
                            licitacion = ET.SubElement(licitaciones,"fecha_publicacion_licitacion")
                        elemento = ET.SubElement(licitacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"

                    elif k == "codigo_poperativo":
                        if programaoperativo == 0:
                            codigos = ET.SubElement(contrato,"programas_operativos")
                            codigo = ET.SubElement(codigos,"programa_operativo")
                            programaoperativo = 1
                        elemento = ET.SubElement(codigo,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"

                    elif k == "cod_medio_pub-formalizacion" or k == "f_publicacion-formalizacion" or k == "descripcion-formalizacion":
                        if fechasformalizacion == 0:
                            formalizaciones = ET.SubElement(contrato,"fechas_publicacion_formalizacion")
                            formalizacion = ET.SubElement(formalizaciones,"fecha_publicacion_formalizacion")
                            fechasformalizacion = 1
                        k = k.split("-")[0]
                        elemento = ET.SubElement(formalizacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"

                    elif k == "cod_subcontratista" or k == "titulo_contrato" or k == "importe":
                        if subcontratistascount == 0:
                            subcontratistas = ET.SubElement(contrato,"subcontratistas")
                            subcontratista = ET.SubElement(subcontratistas,"subcontratista")
                            subcontratistascount = 1
                        elemento = ET.SubElement(formalizacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"

                    elif k == "cod_org_adjudicatario":
                        if utescount == 0:
                            utes = ET.SubElement(contrato,"utes")
                            ute = ET.SubElement(utes,"ute")
                            subcontratistascount = 1
                        elemento = ET.SubElement(formalizacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
                    elif k == "cod_adjudicatario":
                        elemento = ET.SubElement(contrato,f"{k}")
                        elemento.text= buscaorg(v,dictionary)
                    else:
                        elemento = ET.SubElement(contrato,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
        tree = ET.ElementTree(root)
        tree.write("C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/Contratos/CONTRATOS_CEUTA.xml", encoding = "UTF-8", xml_declaration=True,)
        
        

In [7]:
def transacciones(file):
    root = ET.Element("transacciones")
    with open(file) as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        for row in csv_reader:
            #print(row)
            transacion = ET.SubElement(root,"transaccion")
            transaccionescount = 0
            for k,v in row.items():
                if v == "":
                    pass
                else:
                    k = k.split(" ")[0]
                    if k == "cod_contrato" or k == "version_cont" or k=="gasto_abonado":
                        if transaccionescount == 0:
                            transacciones = ET.SubElement(transacion,"transacciones_contratos")
                            transaccion2 = ET.SubElement(transacciones,"transaccion_contratos")
                            transaccionescount = 1
                        elemento = ET.SubElement(transaccion2,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
                    else:
                        elemento = ET.SubElement(transacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
    tree = ET.ElementTree(root)
    tree.write("C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/Ceuta_TB1.xml", encoding = "UTF-8", xml_declaration=True,)
        

In [8]:
def listastransacciones(file, tipo):
    root = ET.Element("transaccionesListas")
    with open(file) as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        registros = 0
        ficheros = 1
        for row in csv_reader:
            #print(row)
            #cod_poperativo = row["cod_poperativo"]
            #operacion = row["cod_operacion"]
            #transaccion = row["cod_transaccion"]
            #preguntas = 1
            with open("C:/Users/bllanos/Desktop/project1/prueba1/Julio_2021/Listas_ver/Transacciones/tabla_respuestas/respuestas.csv") as r:
                respuestasreader = csv.DictReader(r,delimiter=";")
                for line in respuestasreader:
                    transaccion = ET.SubElement(root,"transaccionLista")
                    poperativo = ET.SubElement(transaccion,"cod_poperativo")
                    poperativo.text = row["cod_poperativo"]
                    operacion = ET.SubElement(transaccion,"cod_operacion")
                    operacion.text = row["cod_operacion"]
                    codtransaccion = ET.SubElement(transaccion,"cod_transaccion")
                    codtransaccion.text = row["cod_transaccion"]
                    tipolista = ET.SubElement(transaccion,"cod_tipo_lista")
                    tipolista.text = "Va-Tr"
                    lista = ET.SubElement(transaccion,"cod_lista")
                    lista.text = "VaTr0"
                    pregunta = ET.SubElement(transaccion,"cod_pregunta")
                    pregunta.text = line["pregunta"]
                    respuesta = ET.SubElement(transaccion,"respuesta")
                    respuesta.text = line["respuesta"]
                    registros +=1
                if registros > 15000:
                    tree = ET.ElementTree(root)
                    archivo = f"C:/Users/bllanos/Desktop/project1/prueba1/Transacciones_positivas/Vacunas/Ceuta_T{tipo}_LV_{ficheros}.xml"
                    tree.write(archivo, encoding = "UTF-8", xml_declaration=True,)
                    ficheros +=1
                    registros = 0  
                    root = ET.Element("transaccionesListas")
        tree = ET.ElementTree(root)
        tree.write(f"C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/Ceuta_T{tipo}_LV_{ficheros}.xml", encoding = "UTF-8", xml_declaration=True,)


In [9]:
def asociacioncontratos(file):
    with open(file,"r") as f:
        reader = csv.reader(f)
        f.readline()
        dic = {}
        opdic= {}
        loop= 0 
        for line in f.readlines():
            data = line.split(";")
            prog_op = data[0]
            operacion = data[1]
            contrato = data[-3]
            try:
                opdic[prog_op].append(contrato)
            except KeyError:
                opdic.update({prog_op:[contrato]})
            dic.update({operacion:opdic})
                            
    return dic    

In [10]:
def asigacontratos(diccionario):
    root = ET.Element("operacionesModif")
    for k,v in diccionario.items():
        operacion = k
        for prog, conts in v.items():
            programa = prog
            operacion = ET.SubElement(root,"operacion")
            elemento = ET.SubElement(operacion,"cod_poperativo")
            elemento.text = prog
            elemento = ET.SubElement(operacion,"cod_operacion")
            elemento.text = k
            elemento = ET.SubElement(operacion,"contratos_publicos")
            elemento.text = "S"
            contratos = ET.SubElement(operacion,"contratos")
            for item in conts:
                contrato = ET.SubElement(contratos,"contrato")
                elemento = ET.SubElement(contrato,"cod_contrato")
                elemento.text = item
        
    tree = ET.ElementTree(root)
    tree.write("C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/asignacioncontratos.xml", encoding = "UTF-8", xml_declaration=True,)

In [11]:
def transaccionesB2(file):
    root = ET.Element("transacciones")
    with open(file) as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        for row in csv_reader:
            #print(row)
            transacion = ET.SubElement(root,"transaccion")
            transaccionescount = 0
            for k,v in row.items():
                if v == "":
                    pass
                else:
                    k = k.split(" ")[0]
                    if k == "cod_contrato" or k == "version_cont" or k=="gasto_abonado":
                        if transaccionescount == 0:
                            transacciones = ET.SubElement(transacion,"transacciones_contratos")
                            transaccion2 = ET.SubElement(transacciones,"transaccion_contratos")
                            transaccionescount = 1
                        elemento = ET.SubElement(transaccion2,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
                    else:
                        elemento = ET.SubElement(transacion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
    tree = ET.ElementTree(root)
    tree.write("C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/transacciones_pos/Ceuta_B2.xml", encoding = "UTF-8", xml_declaration=True,)
        

In [12]:
def transaccionesIP(file):
    root = ET.Element("transacciones_ips")
    with open(file) as f:
        csv_reader = csv.DictReader(f,delimiter=";")
        for row in csv_reader:
            #print(row)
            transaccion = ET.SubElement(root,"transaccion_ip")
            indicadorescount = 0
            codigoscount = 0
            #fechasformalizacion = 0
            #subcontratistascount = 0 
            #utescount = 0
            for k,v in row.items():
                #print(k,v)
                if v == "":
                    pass
                else:
                    #v = ajustanumeros(v)
                    k = k.split(" ")[0]
                    if k == "cod_ind_productividad" or k == "fecha" or "val" in k or k == "observaciones":
                        if k == "cod_ind_productividad":
                            codigoscount += 1
                            if indicadorescount == 0:
                                indicadores = ET.SubElement(transaccion,"indicadores_productividad")
                                indicador = ET.SubElement(indicadores,"indicador_productividad")
                                indicadorescount = 1
                            else:
                                indicador = ET.SubElement(indicadores,"indicador_productividad")
                        elemento = ET.SubElement(indicador,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"

                    else:
                        elemento = ET.SubElement(transaccion,f"{k}")
                        elemento.text= f"{v.replace(',','.')}"
        tree = ET.ElementTree(root)
        tree.write("C:/Users/bllanos/Desktop/project1/prueba1/Dec_2021/Ceuta/Transacciones_IP/CEU_CV6-CV40_dec.xml", encoding = "UTF-8", xml_declaration=True,)

In [13]:
organismosdic = orgs_dic(archivo_organismos)

In [14]:
contratos(contratosfile,organismosdic)

In [15]:
transacciones(transaccionesfile)
listastransacciones(transaccionesfile,"B1")

In [16]:
asigacontratos(asociacioncontratos(transaccionesfile))

In [17]:
transaccionesB2(transaccionesB2file)
listastransacciones(transaccionesB2file,"B2")

In [18]:
transaccionesIP(transaccionesIPfile)